In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import acquire

## Acquire and Summarize

### EXERCISE ONE
Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [3]:
df = acquire.acquire_cache_zillow()

The `acquire_cache_zillow` function achieves the goals of exercise one through the following query:

```
SELECT prop.*, logerror, transactiondate
FROM properties_2017 as prop 
INNER JOIN (
	SELECT id, p.parcelid, logerror, transactiondate
	FROM predictions_2017 AS p
	INNER JOIN (
		SELECT parcelid,  MAX(transactiondate) AS max_date
		FROM predictions_2017 
		GROUP BY (parcelid)) AS sub
			ON p.parcelid = sub.parcelid
		WHERE p.transactiondate = sub.max_date
		) AS subq
	ON prop.id = subq.id
WHERE latitude IS NOT NULL 
	AND longitude IS NOT NULL
	AND transactiondate BETWEEN '2017-01-01' AND '2017-12-31'
```

In [4]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN,0.025595,2017-01-01
1,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,0.055619,2017-01-01
2,2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,...,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN,0.005383,2017-01-01
3,3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,...,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN,-0.103410,2017-01-01
4,4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,...,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN,0.006940,2017-01-01


In [6]:
df.shape

(77413, 61)